# Домашнее задание.
### Работа с BigQuery с помощью SQL запросов

Сначала подготовим среду для работы с BigQuery, далее к решению предлагаются задачи

In [4]:
import os

# тут должен быть путь к вашему ключу
import pandas as pd

os.environ["GOOGLE_APPLICATION_CREDENTIALS"]= \
"loftschoolstudy-cf739a23cdf0.json"

In [2]:
# Для уставновки пакетов, используйте следующующие команды
!pip install --upgrade google-cloud-core
!pip install --upgrade google-cloud-storage
!pip install --upgrade google-cloud-bigquery
!pip install --upgrade google-api-python-client
!pip install --upgrade google.cloud.bigquery==2.34.1
!pip install google.cloud.storage
!pip install conda
!pip install google
!pip install pandas
!pip install pyarrow


# Если в ходе решения вам понадобится установка дополнительных библиотек, 
# устанавливайте их с помощью pip или conda
# Не стесняйтесь копировать текст ошибок прямиком в поисковик!

  Using cached google_cloud_bigquery-3.0.1-py2.py3-none-any.whl (210 kB)
  Attempting uninstall: google-cloud-bigquery
    Found existing installation: google-cloud-bigquery 2.34.1
    Uninstalling google-cloud-bigquery-2.34.1:
      Successfully uninstalled google-cloud-bigquery-2.34.1
  Using cached google_cloud_bigquery-2.34.1-py2.py3-none-any.whl (206 kB)
  Attempting uninstall: google.cloud.bigquery
    Found existing installation: google-cloud-bigquery 3.0.1
    Uninstalling google-cloud-bigquery-3.0.1:
      Successfully uninstalled google-cloud-bigquery-3.0.1


  error: subprocess-exited-with-error
  
  python setup.py egg_info did not run successfully.
  exit code: 1
  
  [18 lines of output]
  
  as a python library.  You are not installing a conda executable command
  or activate/deactivate commands.  If your intention is to install conda
  as a standalone application, currently supported install methods include
  the Anaconda installer and the miniconda installer.  If you'd still like
  for setup.py to create entry points for you, use `utils/setup-testing.py`.
  
  Traceback (most recent call last):
    File "<string>", line 2, in <module>
    File "<pip-setuptools-caller>", line 34, in <module>
    File "C:\Users\bkris\AppData\Local\Temp\pip-install-27amnx5v\conda_f950c865c1804920ba3229643fced24d\setup.py", line 35, in <module>
      import conda._vendor.auxlib.packaging  # NOQA
    File "C:\Users\bkris\AppData\Local\Temp\pip-install-27amnx5v\conda_f950c865c1804920ba3229643fced24d\conda\__init__.py", line 13, in <module>
      from .comm

  Using cached conda-4.3.16.tar.gz (299 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'


Для части задач будем использовать датасет `chicago_crime`

In [55]:
from google.cloud import bigquery

client = bigquery.Client()

dataset_ref = client.dataset("chicago_crime", project="bigquery-public-data")

dataset = client.get_dataset(dataset_ref)

### Задача 1

Напишите код, необходимый для подсчета числа таблиц в датасете Chicago Crime

In [56]:
tables = list(client.list_tables(dataset))
print(len(tables))

1


### Задача 2

Как много колонок в таблице `crime` имеют тип `TIMESTAMP`?

In [57]:
table_reference = dataset_ref.table('crime')
table = client.get_table(table_reference)

number_of_timestamp_fields = 0
for column in table.schema:
      if column.field_type == 'TIMESTAMP':
            number_of_timestamp_fields += 1

print(number_of_timestamp_fields)


2


### Задача 3.

Для того чтобы была возможность отобразить преступления на карте, нужно создать массив из пар координат. Получите эти данные из таблицы и сохраните их в списке

In [88]:
import pandas

# библиотека db_dtypes не устанавливается корректно, поэтому просто приводим тип к pandas DataFrame
fields_for_plotting = pandas.DataFrame(client.list_rows(table, selected_fields=table.schema[-3:-1]))
print(fields_for_plotting)


                                     0
0        (41.705329574, -87.701312543)
1         (41.80171934, -87.630703621)
2        (41.974129858, -87.766357256)
3        (41.834011289, -87.635219909)
4        (41.794509508, -87.633301738)
...                                ...
7529159  (41.985300429, -87.662803281)
7529160  (41.985610724, -87.663307609)
7529161   (41.99437391, -87.673902188)
7529162  (41.990874413, -87.663833604)
7529163  (41.989532092, -87.674673125)

[7529164 rows x 1 columns]


Далее работаем с датасетом `openaq` с таблицей `global_air_quality`

In [23]:
from google.cloud import bigquery

# Create a "Client" object
client = bigquery.Client()

dataset_ref = client.dataset("openaq", project="bigquery-public-data")

dataset = client.get_dataset(dataset_ref)

table_ref = dataset_ref.table("global_air_quality")

table = client.get_table(table_ref)

print(pandas.DataFrame(client.list_rows(table, max_results=5)))

                                                   0
0  (Near Road, Providence-New Bedford-Fall River,...
1  (Near Road, Providence-New Bedford-Fall River,...
2  (Near Road, Providence-New Bedford-Fall River,...
3  (Near Road, Providence-New Bedford-Fall River,...
4  (Near Road, Providence-New Bedford-Fall River,...


### Задача 4

Напишите запрос к таблице, в результате которого мы узнаем,
какие страны используют единицы `ppm` для измерения загрязненности воздуха.

Напишите свой запрос в ячейке ниже, и поместите в переменную `first_query`.

Напомню, как выглядят запросы такого типа:
```SQL
query = """
        SELECT city
        FROM `bigquery-public-data.openaq.global_air_quality`
        WHERE country = 'US'
        """
```

In [96]:
first_query = """SELECT DISTINCT(country)
                 FROM `bigquery-public-data.openaq.global_air_quality`
                 WHERE unit = 'ppm'
                 """

safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
first_query_job = client.query(first_query, job_config=safe_config)

first_results = pandas.DataFrame(first_query_job)

print(first_results.head())

      0
0  (US)
1  (PE)
2  (AU)
3  (ZA)
4  (MX)


### Задача 5

В каких местах уровень загрязнения воздуха равен нулю?
1. Определите запрос `zero_pollution_query`, выбирающий все колонки таблицы,
где `value` равен нулю
2. В переменнную `zero_pollution_results` сохраните `pd.DataFrame`
этой отфильтрованной таблицы

In [97]:
zero_pollution_query = """SELECT *
                          FROM `bigquery-public-data.openaq.global_air_quality`
                          WHERE value = 0"""

safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
query_job = client.query(zero_pollution_query, job_config=safe_config)

zero_pollution_results = pandas.DataFrame(query_job)

print(zero_pollution_results.head())

                                                   0
0  (HU-Beltsville, Washington-Arlington-Alexandri...
1  (HU-Beltsville, Washington-Arlington-Alexandri...
2  (HU-Beltsville, Washington-Arlington-Alexandri...
3  (HU-Beltsville, Washington-Arlington-Alexandri...
4  (HU-Beltsville, Washington-Arlington-Alexandri...


Теперь рассмотрим таблицу комментариев из датасета новостного портала `hacker_news`

In [98]:
from google.cloud import bigquery

client = bigquery.Client()

dataset_ref = client.dataset("hacker_news", project="bigquery-public-data")

dataset = client.get_dataset(dataset_ref)

table_ref = dataset_ref.table("comments")

table = client.get_table(table_ref)

client.list_rows(table, max_results=5)

### Задача 6

Хакер-ньюс разыгрывают премию между авторами, у которых более 10 тысяч новостей.
Давайте, с помощью SQL запроса, найдем всех авторов с более 10_000 постов и количество их постов.
Назовите колонку с количеством постов `NumPosts`

Здесь нам пригодится оператор `GROUP BY ... HAVING`, пример запроса:

```SQL
query = """
        SELECT parent, COUNT(1) AS NumPosts
        FROM `bigquery-public-data.hacker_news.comments`
        GROUP BY parent
        HAVING COUNT(1) > 10
        """
        ```

In [104]:
prolific_commenters_query = """SELECT author,
                               COUNT(1) as NumPosts
                               FROM `bigquery-public-data.hacker_news.comments`
                               GROUP BY author
                               HAVING COUNT(1) > 10000"""

safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
query_job = client.query(prolific_commenters_query, job_config=safe_config)

prolific_commenters = pandas.DataFrame(query_job)

print(prolific_commenters.head())

                       0
0  (dragonwriter, 10723)
1         (None, 227736)
2           (eru, 10448)
3       (rbanffy, 10557)
4         (DanBC, 12902)


### Задача 7. 

Как много комментариев в таблице удалено? Используйте поле `deleted` со значением `True`, чтобы вывести суммарное число удаленных комментариев

In [110]:
deleted_posts_query = """SELECT COUNT(*)
                               FROM `bigquery-public-data.hacker_news.comments`
                               WHERE deleted=True"""

safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
query_job = client.query(deleted_posts_query, job_config=safe_config)

deleted_posts = pandas.DataFrame(query_job)

print(deleted_posts.head())

num_deleted_posts = deleted_posts[0]

          0
0  (227736)


Следующий датасет к рассмотрению --- `world_bank_intl_education` и таблица `international_education`

In [6]:
from google.cloud import bigquery

client = bigquery.Client()

dataset_ref = client.dataset("world_bank_intl_education", project="bigquery-public-data")

dataset = client.get_dataset(dataset_ref)

table_ref = dataset_ref.table("international_education")

table = client.get_table(table_ref)

client.list_rows(table, max_results=5)

В этой таблице значения в колонке `indicator_code` обозначают, что за данные представлены в строке.

Например, индикатор `SE.XPD.TOTL.GD.ZS` отвечает за `Государственные расходы на образование в процентах от ВВП (%)`

### Задача 8

Напишите запрос, в результате которого будет видно, какие страны тратят наибольшую долю ВВП на образование?

Сначала нужно выделить все строки, где `SE.XPD.TOTL.GD.ZS`, и напишите запрос, который
вернет среднее значение колонки `value` для каждой страны в датасете, в период 2010-2017 гг
(обе границы включительно)

Требования:
* Результаты долдны иметь имена стран, а не коды стран. Для каждой страны одна строка в результате
* Используйте имя `avg_ed_spending_pct` для колонки, которую создадите с помощбю `AVG()`
* Упорядочьте результаты так, чтобы страны с наибольшим средним отображались первыми. 

Пример похожего запроса на другом датасете:

```SQL
query = """
        SELECT COUNT(consecutive_number) AS num_accidents, 
               EXTRACT(DAYOFWEEK FROM timestamp_of_crash) AS day_of_week
        FROM `bigquery-public-data.nhtsa_traffic_fatalities.accident_2015`
        GROUP BY day_of_week
        ORDER BY num_accidents DESC
        """
        ```

In [118]:
country_spend_pct_query = """
                          SELECT country_name,
                          AVG(value) as avg_ed_spending_pct
                          FROM `bigquery-public-data.world_bank_intl_education.international_education`
                          WHERE indicator_code = 'SE.XPD.TOTL.GD.ZS' AND
                          year BETWEEN 2010 and 2017
                          GROUP BY country_name
                          ORDER BY 2 DESC
                          """

safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
country_spend_pct_query_job = client.query(country_spend_pct_query, job_config=safe_config)

country_spending_results = pandas.DataFrame(country_spend_pct_query_job)

print(country_spending_results.head())

                                           0
0                     (Cuba, 12.83726978302)
1  (Micronesia, Fed. Sts., 12.4677495956421)
2        (Solomon Islands, 10.0010795593262)
3                (Moldova, 8.37215256690979)
4                (Namibia, 8.34961032867432)
                                           0
0                     (Cuba, 12.83726978302)
1  (Micronesia, Fed. Sts., 12.4677495956421)
2        (Solomon Islands, 10.0010795593262)
3                (Moldova, 8.37215256690979)
4                (Namibia, 8.34961032867432)


### Задача 9

В этой таблице тысячи различных кодов в поле `indicator`, чтобы ознакомится с ними
всеми потребуется очень много времени. Множество кодов доступно для ограниченного числа стран,
мы можем отбросить из рассмотрения коды, которые встречаются редко.

Напишите такой запрос, чтобы получить таблицу, содержащую коды и имена `indicator_name`,
для тех `indicator`, которые встречаются более 175 раз в 2016м году.

Требования:

* Одна строка на одно имя и `indicator` код
* Колонки должны называться: `indicator_code`, `indicator_name` и `num_rows`
* Результат должен содержать только коды, где >= 175 строк
* Чтобы получить и `indicator_code`, и `indicator_name` в результирующем DataFrame, нужно добавить оба поля в `SELECT` (в дополнении к агрегации `COUNT()`). Это значит, что их оба нужно добавить и в `GROUP BY`
* Упорядочьте результаты в порядке убывания (сверху вниз)

In [16]:
import pandas
code_count_query = """SELECT indicator_name,
                        indicator_code,
                        COUNT(*)
                        FROM `bigquery-public-data.world_bank_intl_education.international_education`
                        GROUP BY indicator_code, indicator_name
                        HAVING COUNT(*) >= 175
                        ORDER BY 3 DESC"""

safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
code_count_query_job = client.query(code_count_query, job_config=safe_config)

code_count_results = pandas.DataFrame(code_count_query_job)

print(code_count_results.head())

                                                   0
0            (Population, total, SP.POP.TOTL, 11155)
1  (Population growth (annual %), SP.POP.GROW, 11...
2  (Population, ages 15-64 (% of total), SP.POP.1...
3  (Population, male (% of total), SP.POP.TOTL.MA...
4  (Population, ages 0-14 (% of total), SP.POP.00...


Наконец, перейдем к `JOIN`. Работать будем, как вы могли догадаться, с новым датасетом.
С таблицами `posts_questions` и `posts_questions` из датасета `stackoverflow`


In [17]:
from google.cloud import bigquery

lient = bigquery.Client()

dataset_ref = client.dataset("stackoverflow", project="bigquery-public-data")

dataset = client.get_dataset(dataset_ref)

table_ref = dataset_ref.table("posts_questions")

table = client.get_table(table_ref)

client.list_rows(table, max_results=5)

Таблица ответов

In [18]:
table_ref = dataset_ref.table("posts_answers")

table = client.get_table(table_ref)

client.list_rows(table, max_results=5)

### Задача 10

Как долго приходится ждать, чтобы получить ответ на вопрос? 

Рассмотрим запрос, с его помощью получаем информацию по вопросам, заданным в 2018 году. Он возвращает:
* `q_id` идентификатор вопроса
* `time_to_answer` сколько секунд прошло до ответа на вопрос

Ячейку ниже нужно выполнить не изменяя

In [22]:
first_query = """
              SELECT q.id AS q_id,
                  MIN(TIMESTAMP_DIFF(a.creation_date, q.creation_date, SECOND)) as time_to_answer
              FROM `bigquery-public-data.stackoverflow.posts_questions` AS q
                  INNER JOIN `bigquery-public-data.stackoverflow.posts_answers` AS a
              ON q.id = a.parent_id
              WHERE q.creation_date >= '2018-01-01' and q.creation_date < '2018-02-01'
              GROUP BY q_id
              ORDER BY time_to_answer
              """

first_result = client.query(first_query).result()
print("Процент отвеченных вопросов: %s%%" % \
      (sum(first_result["time_to_answer"].notnull()) / len(first_result) * 100))
print("Количество вопросов:", len(first_result))
first_result.head()

134731

Видим подозрительные результаты. Очевидно, с запросом что-то не так.

* Исходя из запроса, 100% вопросов в 2018 получили ответы. Но статистика сайта говорит,
что только 80% всех вопросов бывают с ответами
* Полное число вопросов удивительно мало 134 тысячи. 

Учитывая проблемы, указанные выше, кажется что, выбранный тип `JOIN` отфильтровал неотвеченные вопросы и сохранил лишь те, что с ответами. Предлагается изменить запрос так, чтобы достичь желаемого поведения, использую один из методов для джойна таблиц (`INNER, LEFT, RIGHT` или `FULL`)

In [21]:
correct_query = """
              SELECT q.id AS q_id,
                  MIN(TIMESTAMP_DIFF(a.creation_date, q.creation_date, SECOND)) as time_to_answer
              FROM `bigquery-public-data.stackoverflow.posts_questions` AS q
                  LEFT JOIN `bigquery-public-data.stackoverflow.posts_answers` AS a
              ON q.id = a.parent_id
              WHERE q.creation_date >= '2018-01-01' and q.creation_date < '2018-02-01'
              GROUP BY q_id
              ORDER BY time_to_answer
              """

first_result = client.query(correct_query).result()
print("Процент отвеченных вопросов: %s%%" % \
      (sum(first_result["time_to_answer"].notnull()) / len(first_result) * 100))
print("Количество вопросов:", len(first_result))
first_result.head()

# to_dataframe не работает, но я посмотрела на длину возвращаемого запроса и увидела, что она больше, чем длина
# датафрейма на прошлой итерации.

161837